<div class="alert alert-block alert-info">
<b>Deadline:</b> March 23, 2022 (Wednesday) 23:00
</div>

# Exercise 2. Recommender system

In this exercise, your task is to design a recommender system.

## Learning goals:
* Practise tuning a neural network model by using different regularization methods.

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as Data

import tools
import data

In [2]:
skip_training = True  # Set this flag to True before validation and submission

In [3]:
# During evaluation, this cell sets skip_training to True
# skip_training = True

import tools, warnings
warnings.showwarning = tools.customwarn

In [4]:
# When running on your own computer, you can specify the data directory by:
# data_dir = tools.select_data_dir('/your/local/data/directory')
data_dir = tools.select_data_dir()

The data directory is /coursedata


In [5]:
# Select the device for training (use GPU if you have one)
#device = torch.device('cuda:0')
device = torch.device('cpu')

In [6]:
if skip_training:
    # The models are always evaluated on CPU
    device = torch.device("cpu")

## Ratings dataset

We will train the recommender system on the dataset in which element consists of three values:
* `user_id` - id of the user (the smallest user id is 1)
* `item_id` - id of the movie (the smallest item id is 1)
* `rating` - rating given by the user to the item (ratings are integer numbers between 1 and 5.

The recommender system need to predict the rating for any given pair of `user_id` and `item_id`.

We measure the quality of the predicted ratings using the mean-squared error (MSE) loss:
$$
  \frac{1}{N}\sum_{i=1}^N (r_i - \hat{r}_i)^2
$$
where $r_i$ is a real rating and $\hat{r}_i$ is a predicted one.

Note: The predicted rating $\hat{r}_i$ does not have to be an integer number.

In [7]:
trainset = data.RatingsData(root=data_dir, train=True)
testset = data.RatingsData(root=data_dir, train=False)

In [8]:
# Print one sample from the dataset
x = trainset[0]
print(f'user_id={x[0]}, item_id={x[1]}, rating={x[2]}')

user_id=1, item_id=1, rating=5


# Model

You need to design a recommender system model with the API described in the cell below.

Hints on the model architecture:
* You need to use [torch.nn.Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html?highlight=embedding#torch.nn.Embedding) layer to convert inputs `user_ids` and `item_ids` into reasonable representations. The idea of the embedding layer is that we want to represent similar users with values that are close to each other. The original representation as integers is not good for that. By using the embedding layer, we can learn such useful representations automatically.

### Model tuning

In this exercise, you need to tune the architecture of your model to achieve the best performance on the provided test set. You will notice that overfitting is a severe problem for this data: The model can easily overfit the training set producing poor accuracy on the out-of-training (test) data.

You need to find an optimal combination of the hyperparameters, with some hyperparameters corresponding to the regularization techniques that we studied in the lecture.

The hyperparameters that you are advised to consider:
* Learning rate value and learning rate schedule (decresing the learning rate often has positive effect on the model performance)
* Number of training epochs
* Network size
* Weight decay
* Early stopping
* Dropout
* Increase amount of data:
  * Data augmentation
  * Injecting noise

You can tune the hyperparameters by, for example, grid search, random search or manual tuning. In that case, you can use `architecture` argument to specify the hyperparameters that define the architecture of your network. After you have tuned the hyperparameters, set the default value of this argument to the optimal set of the hyparameters so that the best architecture is used in the accuracy tests.

Note:
* The number of points that you will get from this exercise depends on the MSE loss on the test set:
  * below 1.00: 1 point
  * below 0.95: 2 points
  * below 0.92: 3 points
  * below 0.90: 4 points
  * below 0.89: 5 points
  * below 0.88: 6 points 

In [9]:
class RecommenderSystem(nn.Module):
    def __init__(self, n_users, n_items,
                 architecture=None  # If you want to tune the hyperparameters automatically (e.g. using random
                                    # search), use this argument to specify the hyperparameters that define the
                                    # architecture of your network. After you have tuned the hyperparameters,
                                    # set the default value of this argument to the optimal set of the hyparameters
                                    # so that the best architecture is used in the accuracy tests.
                ):
        """
        Args:
          n_users: Number of users.
          n_items: Number of items.
          architecture: List of
              0. Learning rate schedule : dict of key:epoch item:learning rate
              1. Number of epochs: int
              2. Network size : int
              3. Weight decay : float
              4. Early stopping : tuple of (patience, tolerance)
              5. Dropout : float
              6. Data augmentation : int
              7. Injecting noise : float: std of noise
              8. Embedding dimension : int
        """
        
        super(RecommenderSystem, self).__init__()
        
        self.architecture = architecture if architecture != None else [{0: 0.01, 30: 0.005, 75: 0.002, 100: 0.001}, 125, 5, 0.001, None, 0.1, 0, 0, 100]
    
        self.lr_schedule = self.architecture[0] if self.architecture[0] != None else {0: 0.01, 30: 0.005, 75: 0.002, 100: 0.001}
        
        self.n_epochs = self.architecture[1]
        
        self.network_size = self.architecture[2]
        
        self.weight_decay = self.architecture[3]
        
        self.early_stopping = self.architecture[4]
        
        self.dropout = self.architecture[5]
        
        self.data_augmentation = self.architecture[6]
        self.noise = self.architecture[7]
        
        self.embedding_dim = self.architecture[8]
        
        
        self.user_embedding = torch.nn.Embedding(n_users+1, self.embedding_dim)
        self.item_embedding = torch.nn.Embedding(n_items+1, self.embedding_dim)
        
        self.fc_layers = torch.nn.ModuleList()
        
        self.fc_layers.append(nn.Linear(2 * self.embedding_dim, 150))
        self.fc_layers.append(nn.ReLU())
        if(self.dropout != 0):
            self.fc_layers.append(nn.Dropout(self.dropout))
        
        for i in range(self.network_size):
            self.fc_layers.append(nn.Linear(150, 150))
            self.fc_layers.append(nn.ReLU())
            if(self.dropout != 0):
                self.fc_layers.append(nn.Dropout(self.dropout))
            
        self.fc_layers.append(nn.Linear(150, 50))
        self.fc_layers.append(nn.ReLU())
        if(self.dropout != 0):
                self.fc_layers.append(nn.Dropout(self.dropout))
                
        self.fc_layers.append(nn.Linear(50, 20))
        self.fc_layers.append(nn.ReLU())
        if(self.dropout != 0):
                self.fc_layers.append(nn.Dropout(self.dropout))
                
        self.fc_layers.append(nn.Linear(20, 1))

        
    def forward(self, user_ids, item_ids):
        """
        Args:
          user_ids of shape (batch_size): User ids (starting from 1).
          item_ids of shape (batch_size): Item ids (starting from 1).
        
        Returns:
          outputs of shape (batch_size): Predictions of ratings.
        """
        u = self.user_embedding(user_ids)
        i = self.item_embedding(item_ids)
        
        x = torch.cat([u, i], 1)
        
        for i,_ in enumerate(self.fc_layers):
            x = self.fc_layers[i](x)
                
        x = torch.flatten(x)
        return x
    
class EarlyStopping:
    def __init__(self, tolerance, patience):
        """
        Args:
          patience (int):    Maximum number of epochs with unsuccessful updates.
          tolerance (float): We assume that the update is unsuccessful if the validation error is larger
                              than the best validation error so far plus this tolerance.
        """
        self.tolerance = tolerance
        self.patience = patience
    
    def stop_criterion(self, val_errors):
        """
        Args:
          val_errors (iterable): Validation errors after every update during training.
        
        Returns: True if training should be stopped: when the validation error is larger than the best
                  validation error obtained so far (with given tolearance) for patience epochs (number of consecutive epochs for which the criterion is satisfied).
                 
                 Otherwise, False.
        """
        if len(val_errors) <= self.patience:
            return False

        min_val_error = min(val_errors)
        val_errors = np.array(val_errors[-self.patience:])
        return all(val_errors > min_val_error + self.tolerance)
    

def compute_loss(model, u, i, y):
    model.eval()
    with torch.no_grad():
        outputs = model(u, i)
        loss = F.mse_loss(outputs, y)
        return loss.cpu().numpy()
    
def add_noise(x, noise_std):
    noise = np.random.normal(scale=noise_std, size=np.shape(x))
    return np.add(x, noise).float()

def print_progress(epoch, train_error, val_error):
    print("Epoch: {epoch}, train_error: {train_error}, val_error: {val_error}".format(epoch = epoch, train_error = train_error, val_error = val_error))


You can test the shapes of the model outputs using the function below.

In [10]:
def test_RecommenderSystem_shapes():
    n_users, n_items = 100, 1000
    model = RecommenderSystem(n_users, n_items)
    batch_size = 10
    user_ids = torch.arange(1, batch_size+1)
    item_ids = torch.arange(1, batch_size+1)
    output = model(user_ids, item_ids)
    print(output.shape)
    assert output.shape == torch.Size([batch_size]), "Wrong output shape."
    print('Success')

test_RecommenderSystem_shapes()

torch.Size([10])
Success


In [11]:
# This cell is reserved for testing

## Train the model

You need to train a recommender system using **only the training data.** Please use the test set to select the best model: the model that generalizes best to out-of-training data.

**IMPORTANT**:
* During testing, the predictions are produced by `predictions = model(user_ids, item_ids)` with the `user_ids` and `item_ids` loaded from `RatingsData`.
* There is a size limit of 30Mb for saved models.

In [12]:
# Create the model
# IMPORTANT: the default value of the architecture argument should define your best model.
model = RecommenderSystem(trainset.n_users, trainset.n_items)

In [13]:
# Implement the training loop in this cell
optimizer = torch.optim.Adam(model.parameters(), weight_decay = model.weight_decay)
criterion = nn.MSELoss()

train_loader = Data.DataLoader(
    dataset = trainset,
    batch_size = model.embedding_dim,
    shuffle = True,
    drop_last = True
)
test_loader = Data.DataLoader(
    dataset = testset,
    batch_size = model.embedding_dim,
    shuffle = True,
    drop_last = True
)

if not skip_training:
    n_epochs = model.n_epochs
    train_errors = []  # Keep track of the training data
    val_errors = []  # Keep track of the validation data
    
    if model.early_stopping != None:
        early_stop = EarlyStopping(tolerance=model.early_stopping[0], patience=model.early_stopping[1])

    for epoch in range(n_epochs):
        
        train_errors_loc = []
        for step, (u, i, r) in enumerate(train_loader):
            model.train()
            u = u.to(device)
            i = i.to(device)
            r = r.to(device)


            if epoch in model.lr_schedule.keys():
                for g in optimizer.param_groups:
                    g['lr'] = model.lr_schedule[epoch]


            if model.noise != 0:
                predictions = model(u, i)
            else:
                predictions = model(u, i)

            loss = criterion(predictions.to(torch.float32), r.to(torch.float32))
            optimizer.zero_grad()

            loss.backward()
            optimizer.step()

            train_errors_loc.append(compute_loss(model, u, i, r))
            
        
        val_errors_loc = []
        for step, (u, i, r) in enumerate(test_loader):

            val_errors_loc.append(compute_loss(model, u, i, r))
            
        if model.early_stopping != None and early_stop.stop_criterion(val_errors):
            print(val_errors[epoch])
            print('Stop after %d epochs' % epoch)
            break
        
        train_errors.append(np.mean(train_errors_loc))
        val_errors.append(np.mean(val_errors_loc))
        
        print_progress(epoch, train_errors[epoch], val_errors[epoch])
        
        

Epoch: 0, train_error: 1.4465385675430298, val_error: 1.0166783332824707
Epoch: 1, train_error: 0.9412094354629517, val_error: 0.9543769955635071
Epoch: 2, train_error: 0.8978543281555176, val_error: 0.9584816098213196
Epoch: 3, train_error: 0.8859272599220276, val_error: 0.9577786922454834
Epoch: 4, train_error: 0.8803480267524719, val_error: 0.9321579933166504
Epoch: 5, train_error: 0.8833151459693909, val_error: 0.9332506656646729
Epoch: 6, train_error: 0.8887622356414795, val_error: 0.9797494411468506
Epoch: 7, train_error: 0.8869181871414185, val_error: 0.9549263119697571
Epoch: 8, train_error: 0.8853926062583923, val_error: 0.9450538158416748
Epoch: 9, train_error: 0.8898065090179443, val_error: 0.9902920722961426
Epoch: 10, train_error: 0.8995453119277954, val_error: 0.9648863077163696
Epoch: 11, train_error: 0.8982700109481812, val_error: 0.9476209878921509
Epoch: 12, train_error: 0.9013797044754028, val_error: 0.9698861837387085
Epoch: 13, train_error: 0.8938551545143127, val_

Epoch: 111, train_error: 0.7289848327636719, val_error: 0.8739311099052429
Epoch: 112, train_error: 0.7288282513618469, val_error: 0.8642002940177917
Epoch: 113, train_error: 0.7255070209503174, val_error: 0.8701902627944946
Epoch: 114, train_error: 0.7234106659889221, val_error: 0.8822857737541199
Epoch: 115, train_error: 0.7190868258476257, val_error: 0.8674623966217041
Epoch: 116, train_error: 0.71738600730896, val_error: 0.8679208159446716
Epoch: 117, train_error: 0.7139315009117126, val_error: 0.867800235748291
Epoch: 118, train_error: 0.7126080393791199, val_error: 0.8642305731773376
Epoch: 119, train_error: 0.709266185760498, val_error: 0.8785386681556702
Epoch: 120, train_error: 0.7069787383079529, val_error: 0.8787716627120972
Epoch: 121, train_error: 0.7050453424453735, val_error: 0.8959397077560425
Epoch: 122, train_error: 0.7004585266113281, val_error: 0.8767597675323486
Epoch: 123, train_error: 0.6984192728996277, val_error: 0.8693632483482361
Epoch: 124, train_error: 0.69

In [14]:
# Save the model to disk (the pth-files will be submitted automatically together with your notebook)
# Set confirm=False if you do not want to be asked for confirmation before saving.
if not skip_training:
    tools.save_model(model, 'recsys.pth', confirm=False)

Model saved to recsys.pth.


In [15]:
# This cell loads your best model
if skip_training:
    model = RecommenderSystem(trainset.n_users, trainset.n_items)
    tools.load_model(model, 'recsys.pth', device)

The next cell tests the accuracy of your best model. It is enough to submit .pth files.

**IMPORTANT**:
* During testing, the predictions are produced by `predictions = model(user_ids, item_ids)` with the `user_ids` and `item_ids` loaded from `RatingsData`.
* There is a size limit of 30Mb for saved models. Please make sure that your model loads in the cell above.

In [16]:
# This cell tests the accuracy of your best model.

In [17]:
# This cell is reserved for grading

In [18]:
# This cell is reserved for grading

In [19]:
# This cell is reserved for grading

In [20]:
# This cell is reserved for grading

In [21]:
# This cell is reserved for grading

In [22]:
# This cell is reserved for grading